# SEIR model example
\begin{align}
\dot{S} & = S_f - \alpha\beta\frac{SI}{N+k_I I+k_R R} + r_{R\_S} R\\
\dot{E} & = E_f + \alpha\beta\frac{SI}{N+k_I I+k_R R} - E\frac{1}{t_{E\_I}} \\
\dot{I} & = I_f + E\frac{1}{t_{E\_I}} - I\frac{1}{t_{I\_R}} \\
\dot{R} & = R_f + I\frac{1}{t_{I\_R}} - r_{I\_R} R\\
\end{align}
Where:  
* $S:$ Susceptible
* $E:$ Exposed
* $I:$ Infectious
* $R:$ Removed
* $\alpha:$ Mobilty
* $\beta:$ Infection rate
* $N:$ Total population
* $t_{E\_I}:$ # Transition time between exposed and infectious
* $t_{I\_R}:$ # Transition time between infectious and recovered
* $r_{R\_S}:$ Immunity loss rate ($\frac{1}{t_{R\_S}}$)  
* $S_f,E_f,I_f,R_f:$ External flux
* $k_I:$ Infected saturation   
* $k_R:$ Immunity shield    


In [8]:
# Util libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# cv19 libraries
from cv19gm.models.seir import SEIR

In [9]:
# For pop-up plots execute this code (optional)
import platform
OS = platform.system()

if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')

Linux


# Build a basic SEIR model using a configuration file

## Constant parameters

In [32]:
# Input configuration file
config = 'cfg/SEIR.toml'

In [33]:
# Build simulation object
model = SEIR(config = config)

In [34]:
%%capture
# Simulate (solve ODE)
model.solve()

In [35]:
# Show results
model.results

,t,dates,S,E,E_d,I,I_d,R,R_d,Flux,E_ac,I_ac,R_ac,I_det,I_d_det,I_ac_det,prevalence_total,prevalence_susc,prevalence_det
0,0,None,986000,3000,1500,6000,2999,5000,0,0.0,1500,18000,0,6000,2999,18000,0.018000,0.018000,0.018000
1,1,None,984813,3531,1302,6052,1523,5601,380,0.0,2802,19523,380,6052,1523,19523,0.019523,0.019523,0.019523
2,2,None,983609,3982,1241,6193,1040,6213,527,0.0,4043,20563,908,6193,1040,20563,0.020564,0.020564,0.020564
3,3,None,982372,4382,1240,6401,916,6842,592,0.0,5284,21479,1501,6401,916,21479,0.021480,0.021480,0.021480
4,4,None,981090,4750,1269,6662,918,7495,632,0.0,6553,22398,2133,6662,918,22398,0.022398,0.022398,0.022398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,None,201828,0,0,0,0,798171,0,0.0,786444,806823,793048,0,0,806823,0.806824,0.806824,0.806824
496,496,None,201828,0,0,0,0,798171,0,0.0,786444,806823,793048,0,0,806823,0.806824,0.806824,0.806824
497,497,None,201828,0,0,0,0,798171,0,0.0,786444,806823,793048,0,0,806823,0.806824,0.806824,0.806824
498,498,None,201828,0,0,0,0,798171,0,0.0,786444,806823,793048,0,0,806823,0.806824,0.806824,0.806824


In [36]:
# Plot matplotlib
plt.plot(model.t,model.S,label='S')
plt.plot(model.t,model.E,label='E')
plt.plot(model.t,model.I,label='I')
plt.plot(model.t,model.R,label='R')
plt.xlim(0,300)
plt.legend(loc=0)
plt.show()

In [37]:
# Plot with pandas
ax = model.results.plot(x='t',y=['S','E','I','R'])
ax.set_xlim(0,300)
ax.set_title('SEIR simulation plot')

Text(0.5, 1.0, 'SEIR simulation plot')

## Variable parameters
Create a function with time dependent parameters. Check covid19functions notebook for more information on this.

In [16]:
config2 = 'cfg/SEIR_v2.toml'

In [22]:
model2 = SEIR(config = config2)

In [24]:
%%capture
model2.solve()

In [25]:
plt.plot(model2.t,model2.S,label='S')
plt.plot(model2.t,model2.E,label='E')
plt.plot(model2.t,model2.I,label='I')
plt.plot(model2.t,model2.R,label='R')
plt.xlim(0,300)
plt.legend(loc=0)
plt.show()

# SEIR desde cmd

In [26]:
beta = 0.2
population = 10000
I = 100


In [27]:
model = SEIR(config = config)

In [29]:
model.solve()

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 341
     njev: 14
      nlu: 14
      sol: None
   status: 0
  success: True
        t: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 

In [30]:
model.beta(0)

0.2